In [1]:
# import libraries

import requests
import pandas as pd
import numpy as np

In [2]:
# use FPL API to access FPL data

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
# create dataframes for teams, elements, and element_type data

teams_df = pd.DataFrame(json['teams'])

elements_df = pd.DataFrame(json['elements'])

elements_types_df = pd.DataFrame(json['element_types'])


In [4]:
# filter out irrelevent columns from elements_df

slim_elements_df = elements_df[['id','first_name','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points', 'status']]

# change cost

slim_elements_df['now_cost'] = slim_elements_df['now_cost'] / 10

# map position from element_types_df into slim_elements_df

slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

# map team name from teams_df into slim_elements_df

slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

# ensure all value info is of type int

slim_elements_df['value_season'] = slim_elements_df.value_season.astype(float)

# add points per minute

slim_elements_df['ppg'] = (slim_elements_df['total_points'] / slim_elements_df['minutes']) * 90

# remove players with less than 18 games played minutes

slim_elements_df = slim_elements_df.loc[slim_elements_df.minutes > 0]

# add new value metric: points per minute per cost

slim_elements_df['value'] = slim_elements_df['ppg'] / slim_elements_df['now_cost']

C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Felix\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [5]:
# create new dataframes for each position

fwd_df = slim_elements_df.loc[slim_elements_df.position == 'Forward']
mid_df = slim_elements_df.loc[slim_elements_df.position == 'Midfielder']
def_df = slim_elements_df.loc[slim_elements_df.position == 'Defender']
goal_df = slim_elements_df.loc[slim_elements_df.position == 'Goalkeeper']

In [6]:
# create new dataframes for cheap players in each position

cheap_fwd_df = fwd_df[fwd_df['now_cost'] <= fwd_df['now_cost'].min()+0.5]
cheap_mid_df = mid_df[mid_df['now_cost'] <= mid_df['now_cost'].min()+0.5]
cheap_def_df = def_df[def_df['now_cost'] <= def_df['now_cost'].min()+0.5]
cheap_goal_df = goal_df[goal_df['now_cost'] <= goal_df['now_cost'].min()+0.5]

In [7]:
cheap_fwd_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
514,450,Emmanuel,Dennis,Watford,4,7.4,5.2,245,578886,2.9,15,a,Forward,5.510204,1.059655
519,472,Juan Camilo,Hernández Suárez,Watford,4,1.9,5.0,94,135581,1.6,8,a,Forward,7.659574,1.531915
133,105,Ashley,Barnes,Burnley,4,1.8,5.5,188,43345,0.7,4,a,Forward,1.914894,0.348162
132,104,Jay,Rodriguez,Burnley,4,0.9,5.5,84,19943,0.5,3,a,Forward,3.214286,0.584416
429,516,Joshua,Sargent,Norwich,4,0.1,5.5,43,4125,0.5,3,a,Forward,6.279070,1.141649
517,468,Joshua,King,Watford,4,0.3,5.5,89,5728,0.5,3,a,Forward,3.033708,0.551583
29,530,Folarin,Balogun,Arsenal,4,0.5,5.0,69,24107,0.4,2,a,Forward,2.608696,0.521739
139,111,Matej,Vydra,Burnley,4,0.6,5.4,16,2512,0.4,2,a,Forward,11.250000,2.083333
420,329,Adam,Idah,Norwich,4,0.4,5.0,24,8883,0.4,2,a,Forward,7.500000,1.500000
489,380,Troy,Deeney,Watford,4,2.2,5.4,23,36682,0.4,2,a,Forward,7.826087,1.449275


In [8]:
cheap_mid_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
390,304,Jacob,Murphy,Newcastle,3,0.5,5.0,270,31208,2.6,13,a,Midfielder,4.333333,0.866667
218,171,Allan,Marques Loureiro,Everton,3,2.2,4.5,270,93860,2.7,12,a,Midfielder,4.000000,0.888889
67,77,Christian,Nørgaard,Brentford,3,1.3,5.0,270,94962,2.2,11,a,Midfielder,3.666667,0.733333
184,148,James,McArthur,Crystal Palace,3,0.6,4.5,267,32486,2.4,11,a,Midfielder,3.707865,0.823970
111,70,Yves,Bissouma,Brighton,3,15.6,4.6,270,381688,2.4,11,a,Midfielder,3.666667,0.797101
159,125,Mateo,Kovacic,Chelsea,3,1.0,5.0,206,59220,2.0,10,a,Midfielder,4.368932,0.873786
354,274,Frederico,Rodrigues de Paula Santos,Man Utd,3,0.8,5.0,255,45357,1.8,9,a,Midfielder,3.176471,0.635294
539,421,Declan,Rice,West Ham,3,4.5,5.0,270,78050,1.8,9,a,Midfielder,3.000000,0.600000
469,365,Pierre-Emile,Højbjerg,Spurs,3,2.6,5.0,269,31909,1.6,8,a,Midfielder,2.676580,0.535316
427,471,Pierre,Lees-Melou,Norwich,3,0.1,5.0,252,1236,1.6,8,a,Midfielder,2.857143,0.571429


In [9]:
cheap_def_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
66,76,Pontus,Jansson,Brentford,2,2.8,4.5,270,164933,4.9,22,a,Defender,7.333333,1.629630
471,367,Davinson,Sánchez,Spurs,2,2.5,4.5,270,136665,4.2,19,a,Defender,6.333333,1.407407
319,245,Konstantinos,Tsimikas,Liverpool,2,26.9,4.2,184,795245,4.3,18,a,Defender,8.804348,2.096273
475,371,Japhet,Tanganga,Spurs,2,3.1,4.5,262,205197,3.8,17,a,Defender,5.839695,1.297710
98,56,Shane,Duffy,Brighton,2,12.7,4.1,270,689949,4.1,17,a,Defender,5.666667,1.382114
73,83,Rico,Henry,Brentford,2,1.1,4.5,270,43089,3.1,14,a,Defender,4.666667,1.037037
89,483,Kristoffer,Ajer,Brentford,2,1.0,4.5,250,30734,3.1,14,a,Defender,5.040000,1.120000
270,185,Luke,Ayling,Leeds,2,13.1,4.5,270,252245,2.0,9,a,Defender,3.000000,0.666667
259,219,Luke,Thomas,Leicester,2,1.2,4.5,270,65690,2.0,9,a,Defender,3.000000,0.666667
186,150,Cheikhou,Kouyaté,Crystal Palace,2,0.6,4.5,270,11011,2.0,9,a,Defender,3.000000,0.666667


In [10]:
cheap_goal_df.sort_values('total_points',ascending=False).head(10)

,id,first_name,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,status,position,ppg,value
70,80,David,Raya Martin,Brentford,1,6.4,4.5,270,294276,3.8,17,a,Goalkeeper,5.666667,1.259259
240,200,Kasper,Schmeichel,Leicester,1,19.7,5.0,270,253252,2.6,13,a,Goalkeeper,4.333333,0.866667
182,146,Vicente,Guaita,Crystal Palace,1,5.3,4.5,270,61841,2.7,12,a,Goalkeeper,4.000000,0.888889
350,270,David,de Gea,Man Utd,1,3.8,5.0,270,86480,2.4,12,a,Goalkeeper,4.000000,0.800000
110,69,Robert,Sánchez,Brighton,1,24.0,4.5,270,177634,2.2,10,a,Goalkeeper,3.333333,0.740741
217,170,Jordan,Pickford,Everton,1,6.7,5.0,270,60320,1.8,9,a,Goalkeeper,3.000000,0.600000
400,513,Freddie,Woodman,Newcastle,1,0.6,4.5,270,27865,2.0,9,a,Goalkeeper,3.000000,0.666667
572,475,José,Malheiro de Sá,Wolves,1,0.5,5.0,270,17321,1.6,8,a,Goalkeeper,2.666667,0.533333
284,199,Illan,Meslier,Leeds,1,3.5,5.0,270,21667,1.2,6,a,Goalkeeper,2.000000,0.400000
497,389,Daniel,Bachmann,Watford,1,2.0,4.5,270,18335,1.3,6,a,Goalkeeper,2.000000,0.444444
